In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
生成《中华新韵》十四韵 JSON 文件（基于 3500 常用字）
使用方法：
    1. 安装依赖: pip install pypinyin
    2. 运行: python generate_xinyun.py
    3. 生成 zhonghua_xinyun.json
"""

import json
from pypinyin import lazy_pinyin, Style

# === 从文件读取 3500 常用汉字 ===
try:
    with open("常用汉字表.txt", "r", encoding="utf-8") as f:
        content = f.read()
    # 清理文本
    common_chars = ''.join(ch for ch in content if '\u4e00' <= ch <= '\u9fff')
    common_chars = common_chars[:3500]  # 截取前 3500 字
    print(f"✅ 已加载 {len(common_chars)} 个汉字")
except FileNotFoundError:
    print("❌ 找不到 common_chars_3500.txt，请检查文件是否存在")
    exit()

# 由于文本长度限制，上面是示例。以下是完整 3500 字的获取方式：
# 🌟 我已将完整 3500 字字符串放在最后（可复制粘贴）
# 或：从文件读取（推荐）
# with open("common_chars_3500.txt", "r", encoding="utf-8") as f:
#     common_chars = f.read().replace(" ", "").replace("\n", "").strip()

# === 中华新韵 14 韵部定义（韵母 → 韵部）===
# === 中华新韵 14 韵部定义（韵母 → 韵部）===
yunmu_to_yunbu = {
    # 一麻 a, ia, ua
    'a': '一麻', 'ia': '一麻', 'ua': '一麻',
    # 二波 o, e, uo
    'o': '二波', 'e': '二波', 'uo': '二波',
    # 三皆 ie, üe
    'ie': '三皆', 'üe': '三皆', 've': '三皆',  # 兼容 v
    # 四开 ai, uai
    'ai': '四开', 'uai': '四开',
    # 五微 ei, ui, uei
    'ei': '五微', 'ui': '五微', 'uei': '五微',  
    # 六豪 ao, iao
    'ao': '六豪', 'iao': '六豪',
    # 七尤 ou, iu
    'ou': '七尤', 'iu': '七尤',
    # 八寒 an, ian, uan, üan
    'an': '八寒', 'ian': '八寒', 'uan': '八寒', 'üan': '八寒', 'van': '八寒',
    # 九文 en, in, un, ün
    'en': '九文', 'in': '九文', 'un': '九文', 'ün': '九文', 'vn': '九文',
    # 十唐 ang, iang, uang
    'ang': '十唐', 'iang': '十唐', 'uang': '十唐',
    # 十一庚 eng, ing, ong, iong
    'eng': '十一庚', 'ing': '十一庚', 'ong': '十一庚', 'iong': '十一庚',
    # 十二齐 i, er, ü
    'i': '十二齐', 'er': '十二齐', 'ü': '十二齐', 'v': '十二齐',
    # 十四姑 u, ü
    'u': '十四姑'
}

# 特殊拼音（舌尖音）归“十三支”
zhichi_shiri_zici = {'zhi', 'chi', 'shi', 'ri', 'zi', 'ci', 'si'}

# 初始化韵部
yunbu_list = [
    "一麻", "二波", "三皆", "四开", "五微", "六豪", "七尤", 
    "八寒", "九文", "十唐", "十一庚", "十二齐", "十三支", "十四姑"
]
yun_dict = {name: {"yunmu": [], "zi": []} for name in yunbu_list}

# === 处理每个字 ===
for ch in common_chars:
    if not '\u4e00' <= ch <= '\u9fff':  # 确保是汉字
        continue

    # 获取拼音
    py_list = lazy_pinyin(ch, style=Style.NORMAL)
    if not py_list or not py_list[0]:
        continue
    pinyin_str = py_list[0]

    # 获取韵母
    final_list = lazy_pinyin(ch, style=Style.FINALS)
    if not final_list or not final_list[0]:
        continue
    yunmu = final_list[0]

    # 处理 v → ü
    yunmu = yunmu.replace('v', 'ü').replace('ve', 'üe').replace('vn', 'ün')

    # 判断是否为“十三支”
    if pinyin_str in zhichi_shiri_zici:
        if ch not in yun_dict["十三支"]["zi"]:
            yun_dict["十三支"]["zi"].append(ch)
        continue

    # 查找韵部
    yunbu = yunmu_to_yunbu.get(yunmu)
    if yunbu:
        if ch not in yun_dict[yunbu]["zi"]:
            yun_dict[yunbu]["zi"].append(ch)
        if yunmu not in yun_dict[yunbu]["yunmu"]:
            yun_dict[yunbu]["yunmu"].append(yunmu)
    # else:
    #     print(f"未收录: {ch} -> {pinyin_str} ({yunmu})")

# === 排序 ===
for yunbu in yun_dict:
    yun_dict[yunbu]["zi"].sort()
    yun_dict[yunbu]["yunmu"] = sorted(set(yun_dict[yunbu]["yunmu"]))

# === 保存 JSON ===
output_file = "zhonghua_xinyun.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(yun_dict, f, ensure_ascii=False, indent=2)

print(f"✅ 已生成文件 {output_file}")
print(f"📊 共处理 {len(common_chars)} 个常用汉字")
for yunbu, data in yun_dict.items():
    print(f"   {yunbu}: {len(data['zi'])} 个字")

✅ 已加载 3500 个汉字
✅ 已生成文件 zhonghua_xinyun.json
📊 共处理 3500 个常用汉字
   一麻: 188 个字
   二波: 256 个字
   三皆: 120 个字
   四开: 128 个字
   五微: 186 个字
   六豪: 271 个字
   七尤: 95 个字
   八寒: 482 个字
   九文: 203 个字
   十唐: 245 个字
   十一庚: 373 个字
   十二齐: 395 个字
   十三支: 137 个字
   十四姑: 290 个字
